Thực hiện cào dữ liệu với các thông tin sau:

- Company Link: Đường dẫn của công ty
- Company Name: Tên công ty
- Company Location: Địa điểm của công ty
- Company Scale: Quy mô công ty
- Company Type: Loại hình hoạt động
- Company Website: Website của công ty
- Followers: Người theo dõi
- Company Intro: Giới thiệu của công ty
- Job Name: Tên công việc
- Link : Đường dẫn của job.
- Location: Địa điểm.
- Update Date: Ngày up CV.
- Deadline: Hạn nộp CV.
- Salary: Lương.
- Experience: Kinh nghiệm.
- Rank: Cấp bậc.
- Briefcase: Loại ngành nghề
- Job type: Hình thức làm việc.
- Welfare: Phúc lợi.
- Job Description: Mô tả công việc.
- Job Requirement: Yêu cầu công việc.
- Job Location: Địa điểm làm việc.
- Other Information: Thông tin khác.
- Job Tags: Job Tags

_executable_path_ mọi người tải về chromedriver.exe phù hợp với phiên bản của Chrome đang xài rồi dán đường dẫn chứa chromedriver.exe để cào dữ liệu. Còn trình duyệt khác thì không rõ.

In [ ]:
! pip install selenium
! pip install webdriver_manager

In [1]:
# Necessary Packages 
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
BASE_URL = "https://careerviet.vn/viec-lam/"

# Set up the Chrome Options
service=Service(ChromeDriverManager().install())

In [4]:
def crawlJobLinks(pageStart: int,pageEnd:int) -> list:
    chrome_options=Options()
    prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
    }
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--headless")  # Ví dụ chạy ẩn (tùy chọn)
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=service,options=chrome_options)
    job_links_list=[]
    for page in range(pageStart,pageEnd+1):
        url= f"{BASE_URL}tat-ca-viec-lam-trang-{page}-vi.html"
        driver.get(url)
        time.sleep(random.randint(5, 10))
        for job in driver.find_elements(By.CSS_SELECTOR,'h2 > .job_link'):
            job_links_list.append(job.get_attribute('href'))
        
    driver.quit()
    return job_links_list

In [45]:
def getInfoJob(job_url) -> list:
    chrome_options=Options()
    prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
    }
    chrome_options.add_experimental_option("prefs", prefs)

    chrome_options.add_argument("--headless")  # Ví dụ chạy ẩn (tùy chọn)
    chrome_options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(service=service,options=chrome_options)
    
    driver.get(job_url)
    time.sleep(random.randint(5, 10))
    
    #Tên công việc
    try:
        job_name_element=driver.find_element(By.CSS_SELECTOR,".job-desc h1.title")
        job_name=job_name_element.text
    except:
        job_name="Not Found"
    #Địa điểm
    try:
        location_element=driver.find_element(By.CSS_SELECTOR,".map p a")
        location=location_element.text
    except:
        location="Not Found"

    #Ngày cập nhật
    try:
        update_date=driver.find_element(By.XPATH,"//strong[contains(text(),'Ngày cập nhật')]/following-sibling::p").text
    except:
        update_date="Not Found"

    #Loại ngành nghề
    try:
        li_briefcase_element = driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-briefcase)")
        a_briefcase_elements=li_briefcase_element.find_elements(By.CSS_SELECTOR,"a")
        list_briefcase=""
        for link in a_briefcase_elements:
            list_briefcase+=link.text+", "   
        list_briefcase=list_briefcase.rstrip(", ")
    except:
        list_briefcase="Not Found"

    #Hình thức
    try:
        job_type=driver.find_element(By.XPATH,"//strong[contains(text(),'Hình thức')]/following-sibling::p").text
    except:
        job_type="Not Found"

    #Lương
    try:
        salary=driver.find_element(By.XPATH,"//strong[contains(text(),'Lương')]/following-sibling::p").text
    except:
        salary="Not Found"

    #Kinh nghiệm
    try:
        experience=driver.find_element(By.XPATH,"//strong[contains(text(),'Kinh nghiệm')]/following-sibling::p").text
    except:
        experience="Not Found"

    #Cấp bậc
    try:
        rank=driver.find_element(By.XPATH,"//strong[contains(text(),'Cấp bậc')]/following-sibling::p").text
    except:
        rank="Not Found"

    #Deadline
    try:
        deadline=driver.find_element(By.XPATH,"//strong[contains(text(),'Hết hạn nộp')]/following-sibling::p").text
    except:
        deadline="Not Found"

    #Phúc lợi
    try:
        li_elements=driver.find_elements(By.CSS_SELECTOR,".welfare-list > li")
        list_welfare=""
        for li in li_elements:
            list_welfare+=li.text+", "
        list_welfare=list_welfare.rstrip(", ")
    except:
        job_description="Not Found"

    #Mô tả công việc
    try:
        h2_element=driver.find_element(By.CSS_SELECTOR,".reset-bullet")
        job_description=h2_element.text
    except:
        job_description="Not Found"


    #Yêu cầu công việc
    try:
        h2_element=driver.find_element(By.CSS_SELECTOR,".reset-bullet + .reset-bullet")
        job_requirement=h2_element.text
    except:
        job_requirement="Not Found"

    #Địa điểm làm việc
    try:
        place_element=driver.find_element(By.CSS_SELECTOR,".place-name .place")
        job_location=place_element.text
    except:
        job_location="Not Found"

    #Thông tin khác
    try:
        other_info_element=driver.find_element(By.CSS_SELECTOR,".content_fck")
        other_info=other_info_element.text
    except:
        other_info="Not Found"

    #Thông tin về link công ty
    try:
        company_element=driver.find_element(By.CSS_SELECTOR,"#tabs-job-company a")
        company_url=company_element.get_attribute("href")
    except:
        company_url="Not Found"
    
    #Job tags
    try:
        job_tags_elements=driver.find_elements(By.CSS_SELECTOR,".job-tags li")
        job_tags=""
        for tag in job_tags_elements:
            job_tags+=tag.text+", "
        job_tags=job_tags.rstrip(", ")
    except:
        job_tags="Not Found"

    driver.get(company_url)
    time.sleep(random.randint(5, 10))

    #Tên công ty
    try:
        company_name_element=driver.find_element(By.CSS_SELECTOR,".company-info .content .name")
        company_name=company_name_element.text
    except:
        company_name="Not Found"

    #Địa điểm cụ thể của công ty
    try:
        company_location_element=driver.find_element(By.CSS_SELECTOR,".company-info .content p")
        company_location=company_location_element.text
    except:
        company_location="Not Found"
    
    #Quy mô công ty
    try:
        company_scale_element=driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-account-supervisor)")
        company_scale=company_scale_element.text
        company_scale=company_scale.replace("Qui mô công ty: ","")
    except:
        company_scale="Not Found"
    

    #Loại hình hoạt động
    try:
        company_type_element=driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-gavel)")
        company_type=company_type_element.text
        company_type=company_type.replace("Loại hình hoạt động: ","")
    except:
        company_type="Not Found"
    
    #Đường dẫn công ty
    try:
        company_website_element=driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-link)")
        company_website=company_website_element.text
        company_website=company_website.replace("Website: ","")        
    except:
        company_website="Not Found"

    #Số lượng người theo dõi
    try:
        follow_element=driver.find_element(By.CSS_SELECTOR,"#countFollowers")
        follower=follow_element.text
    except:
        follower="Not Found"

    #Giới thiệu công ty
    try:
        company_intro_element=driver.find_element(By.CSS_SELECTOR,".intro-section .main-text")
        company_intro=company_intro_element.text
    except:
        company_intro="Not Found"
    

    driver.quit()


    job_information=[company_url,company_name,company_location,company_scale,company_type,company_website,follower,company_intro,job_name,job_url,location,update_date,deadline,salary,experience,rank,job_type,list_briefcase,list_welfare,job_description,job_requirement,job_location,other_info,job_tags]

    return job_information 

In [46]:
def crawlJobInformation(job_urls) -> pd.DataFrame:
    job_information_list=[]
    for job_url in job_urls:
        job_information=getInfoJob(job_url)
        job_information_list.append(job_information)
    df=pd.DataFrame(job_information_list,columns=["Company Link","Company Name","Company Location","Company Scale","Company Type","Company Website","Followers","Company Intro","Job Name","Link","Location","Update Date","Deadline","Salary","Experience","Rank","Job Type","Briefcase","Welfare","Job Description","Job Requirement","Job Location","Other Information","Job Tags"])
    return df

In [48]:
df=crawlJobInformation(["https://careerviet.vn/vi/tim-viec-lam/nhan-vien-nghien-cuu-va-phat-trien-san-pham-r-d-thit.35C271C1.html","https://careerviet.vn/vi/tim-viec-lam/nhan-vien-ban-hang-thoi-trang-vincom-my-tho.35C1BE19.html"])

In [49]:
df

,Company Link,Company Name,Company Location,Company Scale,Company Type,Company Website,Followers,Company Intro,Job Name,Link,...,Experience,Rank,Job Type,Briefcase,Welfare,Job Description,Job Requirement,Job Location,Other Information,Job Tags
0,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,CÔNG TY TNHH TM & XNK NHẤT LÂM,"Lô H18 + H19, Ngõ 68, Trung Kính, Yên Hòa",Not Found,Trách nhiệm hữu hạn,http://nhatlamimex.com/,8.503,Công ty TNHH Thương mại và XNK Nhất Lâm hoạt đ...,Nhân Viên Nghiên Cứu Và Phát Triển Sản Phẩm (R...,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,...,3 - 5 Năm,Nhân viên,Nhân viên chính thức,"Thực phẩm & Đồ uống, Công nghệ thực phẩm / Din...","Laptop, Chế độ bảo hiểm, Phụ cấp, Đồng phục, C...",MÔ TẢ CÔNG VIỆC\nNghiên cứu và Phát triển các ...,YÊU CẦU CÔNG VIỆC\nCó kinh nghiệm 2- 3 năm tro...,Hồ Chí Minh,Bằng cấp: Đại học\nĐộ tuổi: 25 - 35\nLương: 15...,nhân viên nghiên cứu và phát triển sản phẩm r&...
1,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,CÔNG TY CP THỜI TRANG LAMER FASHION,"Số 3 , Ngõ 71 Hoàng Văn Thái, Thanh Xuân, Hà Nội",100-499,Not Found,https://lamerfashion.com/,366,Lamer Fashion là thương hiệu thời trang dành c...,Nhân Viên Bán Hàng Thời Trang Vincom Mỹ Tho,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,...,Not Found,Nhân viên,Nhân viên chính thức,"Bán hàng / Kinh doanh, Dệt may / Da giày / Thờ...","Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồng phục, ...",MÔ TẢ CÔNG VIỆC\nTư vấn bán hàng cho KH tại cá...,YÊU CẦU CÔNG VIỆC\nYÊU CẦU CÔNG VIỆC\nGiao tiế...,Tiền Giang,Giới tính: Nữ\nĐộ tuổi: 22 - 45\nLương: 7 Tr -...,"Nhân viên bán hàng, Nhân viên kinh doanh, Sale..."


In [50]:
df.to_csv("sample.csv",index=False,encoding="utf-8-sig")

In [ ]:

df=crawlJobInformation(job_urls)


In [ ]:
df.to_csv("job_information.csv",index=False,encoding="utf-8-sig")